In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [2]:
dimensoes = [
    'porcentagem_domicilios_sem_coleta_de_lixo',
    'porcentagem_domicilios_sem_ligacao_rede_de_agua',
    'porcentagem_domicilios_sem_banheiros',
    'porcentagem_domicilios_sem_esgoto_ou_fossa',
    # # 'porcentagem_domicilios_tipo_comodo',
    'porcentagem_domicilios_cedidos_outra_forma',
    'porcentagem_domicilios_em_outra_condicao_de_ocupacao',
    'numero_de_banheiros_por_habitante',
    'porcentagem_responsaveis_nao_alfabetizado',
    'porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos',
    'porcentagem_responsaveis_renda_ate_3_salarios',
    # # 'porcentagem_responsaveis_menos_8_anos_de_estudo',
    # # 'anos_medios_estudo_responsavel',
    'renda_media_responsavel',
    'numero_de_domicilios_particulares_permanentes',
    'numero_de_domicilios_improvisados',
    'numero_de_pessoas_residentes',
    'porcentagem_de_responsaveis_com_menos_30_anos',
    'numero_medio_pessoas'
]

In [3]:
df = pd.read_csv('resultados/modelo.csv.zip')
df.fillna(0, inplace=True)
# rmsp = (df.rm == 20)
subnormal = (df.tipo_do_setor == 1)
df.loc[:, 'subnormal'] = 0
df.loc[subnormal, 'subnormal'] = 1

/tmp/ipykernel_109424/3615071609.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('resultados/modelo.csv.zip')


In [4]:
r_subnormal = df.loc[(subnormal), ['rm', 'subnormal']].groupby('rm').count().reset_index()
cidades_fora_rms = r_subnormal[(r_subnormal.subnormal < 20) | (r_subnormal.rm == 0.)].rm.to_list()
rms = r_subnormal[(r_subnormal.subnormal >= 20) & (r_subnormal.rm != 0.)].rm.to_list()

In [5]:
def processa_modelos(filter, rm=None, uf=None):

    registro = {}

    # filter = (df.rm == rm)
    df_rm = df.loc[(filter)]

    # print(df_rm.nome_rm.unique()[0])
    if rm:
        registro['nome_rm_ou_UF'] = df_rm.nome_rm.unique()[0]
    
    if uf:
        registro['nome_rm_ou_UF'] = uf
    # print(df_rm.subnormal.value_counts())

    setores_normais = df_rm.loc[filter & (df_rm.loc[filter].subnormal == 0)].shape[0]
    registro['quantidade_original_de_setores_normais'] = setores_normais
    
    setores_subnormais = df_rm.loc[filter & (df_rm.loc[filter].subnormal == 1)].shape[0]
    registro['quantidade_original_de_setores_subnormais'] = setores_subnormais
    print(setores_normais, setores_subnormais)

    samples = min(setores_normais, setores_subnormais)

    # prepara amostras e lições
    x = df_rm.loc[(filter) & (subnormal)]
    x_ = df_rm.loc[(filter) & (~subnormal)].sample(n=samples, random_state=51)
    X = pd.concat([x, x_]).loc[(filter), dimensoes]
    y = pd.concat([x, x_]).loc[(filter), 'subnormal']

    # Processa os modelos, LDA
    # print(y.value_counts())
    clf = LinearDiscriminantAnalysis(solver='lsqr')
    clf.fit(X, y)
    # print(np.unique(clf.predict(X), return_counts=True))
    atribuidos_como_subnormais = np.unique(clf.predict(df_rm.loc[(filter), dimensoes]), return_counts=True)[1][1]
    registro['atribuidos_como_subnormais_LDA'] = atribuidos_como_subnormais
    # print(clf.score(X, y))
    registro['proporcao_atribuido_original_LDA'] = atribuidos_como_subnormais / setores_subnormais
    registro['score_LDA'] = clf.score(X, y)

    # Processa model RandonForest
    clf = RandomForestClassifier(max_depth=10, random_state=0)
    clf.fit(X, y)
    # print(np.unique(clf.predict(X), return_counts=True))
    atribuidos_como_subnormais = np.unique(clf.predict(df_rm.loc[(filter), dimensoes]), return_counts=True)[1][1]
    registro['atribuidos_como_subnormais_RandomForest'] = atribuidos_como_subnormais
    # print(clf.score(X, y))
    registro['proporcao_atribuido_original_RandomForest'] = atribuidos_como_subnormais / setores_subnormais
    registro['score_RandomForest'] = clf.score(X, y)


    # Processa modelo SVC
    clf = LinearSVC(random_state=0, tol=1e-5)
    clf.fit(Normalizer().fit_transform(X), y)
    # print(np.unique(clf.predict(X), return_counts=True))
    atribuidos_como_subnormais = np.unique(clf.predict(Normalizer().fit_transform(df_rm.loc[(filter), dimensoes])), return_counts=True)[1][1]
    registro['atribuidos_como_subnormais_RandomForest'] = atribuidos_como_subnormais
    # print(clf.score(X, y))
    registro['proporcao_atribuido_original_RandomForest'] = atribuidos_como_subnormais / setores_subnormais
    registro['score_RandomForest'] = clf.score(Normalizer().fit_transform(X), y)

    # Salva as predicoes em atributos
    return registro

In [6]:
# Processa cada um dos municípios da sua Região MEtropolitana
result = [] 

for rm in rms:

    rm_filter = (df.rm == rm)
    registro = processa_modelos(rm_filter, rm=rm)
    # Salva as predicoes em atributos
    result.append(registro)

2509 390
995 1057
462 124
1195 337
3929 551
1475 100
973 41
1499 137
3497 1038
1113 201
1025 98
3865 1158
7712 811
907 71
2573 265
16771 2736
26709 3234
2545 423
3840 273
4095 253
1501 27
1079 39
925 25
6622 349
1445 73
5878 181
1360 223


In [7]:
df.columns

Index(['Cod_setor', 'uf', 'nome_UF', 'rm', 'nome_rm', 'meso_regiao',
       'cod_municipio', 'nome_municipio', 'situacao_setor', 'tipo_do_setor',
       'porcentagem_domicilios_sem_coleta_de_lixo',
       'porcentagem_domicilios_sem_ligacao_rede_de_agua',
       'porcentagem_domicilios_sem_banheiros',
       'porcentagem_domicilios_sem_esgoto_ou_fossa',
       'porcentagem_domicilios_cedidos_outra_forma',
       'porcentagem_domicilios_em_outra_condicao_de_ocupacao',
       'numero_de_banheiros_por_habitante',
       'porcentagem_responsaveis_nao_alfabetizado',
       'porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos',
       'porcentagem_responsaveis_renda_ate_3_salarios',
       'renda_media_responsavel',
       'numero_de_domicilios_particulares_permanentes',
       'numero_de_domicilios_improvisados', 'numero_de_pessoas_residentes',
       'porcentagem_de_responsaveis_com_menos_30_anos', 'numero_medio_pessoas',
       'subnormal'],
      dtype='object')

In [8]:
r_ufs = df[df.rm.isin(cidades_fora_rms)].loc[(subnormal), ['nome_UF', 'subnormal']].groupby('nome_UF').count().reset_index()

In [9]:
cidades_fora_rms

[0.0, 6.0, 12.0, 24.0, 28.0, 29.0, 36.0]

In [10]:
r_ufs[r_ufs.subnormal >= 20]

,nome_UF,subnormal
0,AC,33
2,AM,94
3,AP,30
4,BA,39
6,ES,112
8,MG,94
10,PA,127
11,PE,29
12,PR,53
13,RJ,571


In [11]:
# Processa cada uma das UFs
df_uf = df[df.rm.isin(cidades_fora_rms)]
ufs = r_ufs[r_ufs.subnormal >= 20].nome_UF.to_list()

for uf in ufs:

    uf_filter = (df.nome_UF == uf) & (df.rm.isin(cidades_fora_rms))
    print(uf)

    registro = processa_modelos(uf_filter, uf=uf)
    # Salva as predicoes em atributos
    result.append(registro)


AC
841 33
AM
2648 94
AP
194 30
BA
18250 39
ES
3430 112
MG
22740 94
PA
6590 127
PE
7353 29
PR
11897 53
RJ
7691 571
RO
2297 49
RS
15262 99
SC
5639 24
SP2
28883 189


In [12]:
pd.DataFrame(result)

,nome_rm_ou_UF,quantidade_original_de_setores_normais,quantidade_original_de_setores_subnormais,atribuidos_como_subnormais_LDA,proporcao_atribuido_original_LDA,score_LDA,atribuidos_como_subnormais_RandomForest,proporcao_atribuido_original_RandomForest,score_RandomForest
0,RM Manaus,2509,390,1039,2.664103,0.764103,1245,3.192308,0.664103
1,RM Belém,995,1057,1263,1.194891,0.782651,1234,1.167455,0.738304
2,RM Macapá,462,124,175,1.411290,0.899194,215,1.733871,0.596774
3,RM Grande São Luís,1195,337,672,1.994065,0.767062,761,2.258160,0.710682
4,RM Fortaleza,3929,551,1661,3.014519,0.738657,2280,4.137931,0.588929
5,RM Natal,1475,100,666,6.660000,0.755000,765,7.650000,0.565000
6,RM Campina Grande,973,41,210,5.121951,0.890244,448,10.926829,0.646341
7,RM João Pessoa,1499,137,442,3.226277,0.839416,678,4.948905,0.675182
8,RM Recife,3497,1038,2019,1.945087,0.741329,2271,2.187861,0.676301
9,RM Maceió,1113,201,369,1.835821,0.840796,783,3.895522,0.557214
